In [73]:
import os 
import imp
import warnings
import numpy as np
import pandas as pd
import importlib as imp
from tqdm import tqdm, tqdm_notebook
from utils import tools
warnings.simplefilter('ignore')
pd.options.display.max_columns = 100

PATH_TO_BENCHMARKS = './/data//'

**Задача**

*Ваша четвертая лабораторная работа будет заключаться в реализации одного из двух алгоритмов: <br>
метод имитации обжига или генетический алгоритм со встроенной эвристикой в виде локального поиска.  <br>
Алгоритмы реализуются для задачи Cell formation problem.* <br>

Реализовать можно на любом языке. <br>
Также нужно будет сравнить время работы алгоритма, а так же показать лучшее решение алгоритма, которое удалось найти вам. <br>

**Данные:**

Данные для работы прикрепил архивом к письму. **Структура такая:** <br>

m p (number of machines and parts) <br>
Next m rows: <br>
m(row number) list of parts processed by machine m separated by space <br>
e.g: <br>
1 9 17 19 31 33
means machine 1 processes parts 9 17 19 31 33 <br>

**Выходной формат для ответов:** <br>
Output file: instancename.sol (e.g. 20x20.sol) <br>
Output file format: <br>
m1_clusterId m2_clusterId ... - machines to clusters mapping <br>
p1_clusterId p2_clusterId ... - parts to clusters mapping <br>

In [69]:
txtfiles = [f for f in os.listdir(PATH_TO_BENCHMARKS) if os.path.isfile(os.path.join(PATH_TO_BENCHMARKS, f))]
txtfilesfiles

['20x20.txt', '24x40.txt', '30x50.txt', '30x90.txt', '37x53.txt']

In [100]:
tools = imp.reload(tools)
machine_part_matrix = tools.get_data(PATH_TO_BENCHMARKS + txtfiles[0])
machine_part_matrix_sample = np.array([[1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [1, 0, 0, 0, 0], [0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

In [105]:
tools.get_similarity_matrix(machine_part_matrix)

array([[0.        , 0.        , 0.125     , 0.1       , 0.        ,
        0.25      , 0.28571429, 0.25      , 0.25      , 0.        ,
        0.1       , 0.        , 0.09090909, 0.07142857, 0.        ,
        0.3       , 0.        , 0.2       , 0.22222222, 0.33333333],
       [0.        , 0.        , 0.16666667, 0.        , 0.14285714,
        0.        , 0.16666667, 0.14285714, 0.        , 0.33333333,
        0.125     , 0.        , 0.25      , 0.        , 0.125     ,
        0.1       , 0.4       , 0.11111111, 0.        , 0.2       ],
       [0.        , 0.        , 0.        , 0.2       , 0.375     ,
        0.        , 0.11111111, 0.57142857, 0.1       , 0.1       ,
        0.09090909, 0.        , 0.3       , 0.06666667, 0.33333333,
        0.16666667, 0.11111111, 0.08333333, 0.09090909, 0.28571429],
       [0.        , 0.        , 0.        , 0.        , 0.18181818,
        0.18181818, 0.09090909, 0.3       , 0.18181818, 0.18181818,
        0.27272727, 0.14285714, 0.15384615, 0